<a href="https://colab.research.google.com/github/unicamp-dl/IA025_2022S1/blob/main/ex08/Beatriz_Celante/Beatriz_Celante_ex_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = 'Beatriz Celante Vicente'
print(f'Meu nome é {nome}')

Meu nome é Beatriz Celante Vicente


#  Exercício: Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 7, mas iremos agora treinar uma rede neural *com auto-atenção* para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)



O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.9 MB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
     |████████████████████████████████| 6.6 MB 31.8 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
# Check which GPU we are using
!nvidia-smi

Wed May 25 11:54:42 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Implementação do MyDataset

In [6]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, context_size: int):
        self.examples = []
        for text in tqdm_notebook(texts):
            token_ids = tokenize(text=text, tokenizer=tokenizer)
            if len(token_ids) < context_size + 1:
                continue
            # Compute n-grams:
            for i in range(len(token_ids) - context_size):
                input_ids = token_ids[i:i + context_size]
                target_id = token_ids[i + context_size]
                self.examples.append((input_ids, target_id)) 

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        input_ids, target_id = self.examples[idx]
        return torch.LongTensor(input_ids), target_id

## Testando se a implementação do MyDataset está correta

In [7]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")

dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, context_size=3)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
assert len(dummy_dataset) == 5
print('passou no assert de tamanho do dataset')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[ 3396, 10303,   125],
     [ 1660,  5971,   785],
     [ 5971,   785,   125],
     [  785,   125,  1847],
     [  125,  1847, 13779]])

correct_first_batch_target = torch.LongTensor([13239,   125,  1847, 13779, 15616])

assert torch.equal(first_batch_input, correct_first_batch_input)
print('Passou no assert de input')
assert torch.equal(first_batch_target, correct_first_batch_target)
print('Passou no assert de target')

Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/2 [00:00<?, ?it/s]

passou no assert de tamanho do dataset
Passou no assert de input
Passou no assert de target


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [8]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt

--2022-05-25 11:54:47--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula7/sample_brwac.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.74.128, 209.85.147.128, 142.250.125.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.74.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123983611 (118M) [text/plain]
Saving to: ‘sample_brwac.txt’

sample_brwac.txt    100%[===================>] 118.24M   227MB/s    in 0.5s    

2022-05-25 11:54:47 (227 MB/s) - ‘sample_brwac.txt’ saved [123983611/123983611]



In [9]:
# Load datasets
context_size = 9

valid_examples = 100
test_examples = 100
texts = open('sample_brwac.txt').readlines()

print('Truncating for debugging purposes.')
texts = texts[:1000]  

training_texts = texts[:-(valid_examples + test_examples)]
valid_texts = texts[-(valid_examples + test_examples):-test_examples]
test_texts = texts[-test_examples:]

training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, context_size=context_size)
valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, context_size=context_size)
test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, context_size=context_size)

Truncating for debugging purposes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/800 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

In [10]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 927195
valid examples: 130102
test examples: 99902


**Referências utilizadas** \\


https://medium.com/the-dl/transformers-from-scratch-in-pytorch-8777e346ca51 \\
https://towardsdatascience.com/how-to-code-the-transformer-in-pytorch-24db27c8f9ec \\
Conexão residual: \\


In [11]:
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size, context_size, embedding_dim):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            context_size (int): Size of the sequence to consider as context for prediction.
            embedding_dim (int): Dimension of the embedding layer for each word in the context.
        """
        # Escreva seu código aqui.
        super(LanguageModel, self).__init__()

        self.vocab_size = vocab_size
        self.context_size = context_size
        self.embedding_dim = embedding_dim

        #Embedding das palavras - O que elas significam
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        #Positional embedding - Qual a posição das palavras na sentença
        self.positional_embedding = nn.Embedding(context_size, embedding_dim)

        #Projeções lineares
        self.projection_q = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.projection_k = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.projection_v = nn.Linear(embedding_dim, embedding_dim, bias=False)
        self.projection_W0 = nn.Linear(embedding_dim, embedding_dim, bias=False)

        #Fully connected feed-forward network
        self.feed_forward = nn.Sequential(
        nn.Linear(embedding_dim, 2*embedding_dim),
        nn.ReLU(),
        nn.Linear(2*embedding_dim, vocab_size))


    def self_attention(self, q, k, v):
      scores = torch.matmul(q, k.transpose(2,1))
      scores = nn.functional.softmax(scores, dim = -1)
      probs = torch.matmul(scores, v)

      #print(f"scores shape: {scores.shape}")
      #print(f"probs shape: {probs.shape}")

      return probs

    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, context_size)
            
        Returns:
            logits of shape (batch_size, vocab_size)
        """

        #Soma do embedding com o positional encoding - fazemos isso para o transformer entender que a ordem das palavras importam
        x_embed = self.embeddings(inputs) + self.positional_embedding.weight 


        #Criando querry, key e value
        q = self.projection_q(x_embed[:, -1, :]).unsqueeze(1)
        k = self.projection_k(x_embed) 
        v = self.projection_v(x_embed)

        #Verifica se as dimensões estão corretas
        #print(f"X shape: {x_embed.shape}")
        #print(f"Q shape: {q.shape}")
        #print(f"K shape: {k.shape}")
        #print(f"V shape: {v.shape}")



        #Self atention
        out = self.self_attention(q, k, v)
        out = self.projection_W0(out)


        #Reshape
        out = out.reshape((len(inputs),-1))

        #Camada linear (Feed-forward)
        out = self.feed_forward(out)
        return out

## Teste o modelo com um exemplo

In [13]:
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

sample_train, _ = next(iter(DataLoader(training_dataset)))
sample_train_gpu = sample_train.to(device)
model(sample_train_gpu).shape

torch.Size([1, 29794])

In [14]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 11570402


## Assert da Perplexidade


In [15]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, vocab_size)
        target: a LongTensor of shape (batch_size,)

    Returns:
        A float corresponding to the perplexity
    """
    loss = nn.functional.cross_entropy(logits, target, reduction='mean')
    return torch.exp(loss)


n_examples = 1000

sample_train, target_token_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
sample_train_gpu = sample_train.to(device)
target_token_ids = target_token_ids.to(device)
logits = model(sample_train_gpu)

my_perplexity = perplexity(logits=logits, target=target_token_ids)

print(f'my perplexity:              {int(my_perplexity)}')
print(f'correct initial perplexity: {tokenizer.vocab_size}')

assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')

my perplexity:              29761
correct initial perplexity: 29794
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [16]:
max_examples = 100_000_000
eval_every_steps = 10000
lr = 3e-4


model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    context_size=context_size,
    embedding_dim=128,
).to(device)

train_loader = DataLoader(training_dataset, batch_size=32, shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=32)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train_step(input, target):
    model.train()
    model.zero_grad()

    logits = model(input.to(device))
    loss = nn.functional.cross_entropy(logits, target.to(device))
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input, target):
    model.eval()
    logits = model(input)
    loss = nn.functional.cross_entropy(logits, target)
    return loss.item()


train_losses = []
n_examples = 0
step = 0
while n_examples < max_examples:
    for input, target in train_loader:
        loss = train_step(input.to(device), target.to(device)) 
        train_losses.append(loss)
        
        if step % eval_every_steps == 0:
            train_ppl = np.exp(np.average(train_losses))

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(input.to(device), target.to(device))
                    for input, target in validation_loader]))

            print(f'{step} steps; {n_examples} examples so far; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}')
            train_losses = []

        n_examples += len(input)  # Increment of batch size
        step += 1
        if n_examples >= max_examples:
            break

0 steps; 0 examples so far; train ppl: 30177.78, valid ppl: 29903.76
10000 steps; 320000 examples so far; train ppl: 2087.27, valid ppl: 2128.49
20000 steps; 640000 examples so far; train ppl: 1896.60, valid ppl: 2012.43
30000 steps; 960000 examples so far; train ppl: 1846.64, valid ppl: 2036.46
40000 steps; 1280000 examples so far; train ppl: 1779.45, valid ppl: 2056.73
50000 steps; 1600000 examples so far; train ppl: 1786.00, valid ppl: 2008.11
60000 steps; 1920000 examples so far; train ppl: 1791.98, valid ppl: 2048.15
70000 steps; 2240000 examples so far; train ppl: 1729.90, valid ppl: 2074.16
80000 steps; 2560000 examples so far; train ppl: 1765.97, valid ppl: 2062.30
90000 steps; 2880000 examples so far; train ppl: 1745.15, valid ppl: 2116.45
100000 steps; 3200000 examples so far; train ppl: 1691.18, valid ppl: 2147.06
110000 steps; 3520000 examples so far; train ppl: 1718.02, valid ppl: 2098.51
120000 steps; 3840000 examples so far; train ppl: 1681.11, valid ppl: 2174.31
130000 

KeyboardInterrupt: ignored

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [19]:
test_loader = DataLoader(test_dataset, batch_size=64)

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(input.to(device), target.to(device))
        for input, target in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 2407.5909571760294


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [20]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-context_size:]  # Usamos apenas os últimos <context_size> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Eu gosto de comer pizza pois me faz,
Eu gosto de comer pizza pois me faz,,
Eu gosto de comer pizza pois me faz,,,
Eu gosto de comer pizza pois me faz,,,,
Eu gosto de comer pizza pois me faz,,,,,
Eu gosto de comer pizza pois me faz,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,,,
Eu gosto de comer pizza pois me faz,,,,,,,,,,,,,,,,,,,,
